In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("My Decks")

---

Table of Contents <a class="jp-toc-ignore"></a>
=================
* [1 Data loading](#data-loading)
  * [1.1 Read decks](#read-decks)
  * [1.2 Changelog](#changelog)
* [2 Data visualization](#data-visualization)
  * [2.1 Deck composition](#deck-composition)
  * [2.2 Attributes](#attributes)
  * [2.3 Secondary type](#secondary-type)
  * [2.4 Monster type](#monster-type)
  * [2.5 Properties](#properties)
  * [2.6 TCG & OCG Status](#tcg-&-ocg-status)
  * [2.7 Archetypes & Series](#archetypes-&-series)
  * [2.8 ATK & DEF](#atk-&-def)
  * [2.9 Level & Rank](#level-&-rank)
  * [2.10 Link](#link)
  * [2.11 Pendulum scale](#pendulum-scale)
* [3 Check collection](#check-collection)
* [4 Epilogue](#epilogue)
  * [4.1 HTML export](#html-export)
  <!-- * [4.2 Git](#git) -->

# Data loading

## Read decks

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
# Load decks from YDK and decklist files
deck_df = pd.concat([get_ydk(), get_decklists()], ignore_index=True)

# If data loaded
if not deck_df.empty:
    try:
        deck_df = find_cards(deck_df, card_data=True, set_data=True)
    except Exception as e:
        cprint(text=e, color="red")
        cprint(text="Failed to process deck data.", color="red")

# If data not loaded or processing failed. (Page name is always available if data is processed)
if deck_df.empty or "Page name" not in deck_df:
    deck_df, cache_ts = load_latest_data("deck")
    if deck_df is None:
        raise SystemExit("No deck data to process. Aborting!")
    else:
        cprint(text=f"Attempting to proceed with latest deck data from the cache. Timestamp: {cache_ts}", color="yellow")

## Changelog

In [ ]:
# Get latest file if exist
previous_df, previous_ts = load_latest_data("deck")

if previous_df is not None:
    previous_df = previous_df.astype(deck_df[previous_df.columns.intersection(deck_df.columns)].dtypes.to_dict())

In [ ]:
if previous_df is None:
    deck_changelog = None
    print("Skipped")
else:
    deck_changelog = generate_changelog(previous_df, deck_df, col="Name")
    if not deck_changelog.empty:
        display(deck_changelog)
        deck_changelog.to_csv(
            dirs.DATA
            / make_filename(
                report="deck",
                timestamp=timestamp,
                previous_timestamp=previous_ts,
            ),
            index=True,
        )
        print("Changelog saved")

In [ ]:
if deck_changelog is not None and deck_changelog.empty:
    print("No changes. New data not saved")
else:
    deck_df.to_csv(
        dirs.DATA / make_filename(report="deck", timestamp=timestamp),
        index=False,
    )
    print("Data saved")

# Data visualization

In [ ]:
deck_df

## Deck composition

In [ ]:
if (
    "Primary type" in deck_df
    and "Card type" in deck_df
    and not deck_df[["Primary type", "Card type"]].dropna(axis=1, how="all").empty
):
    _ = plot.deck_composition(deck_df)
    plt.show()
else:
    print("No card type data to plot")

## Attributes

In [ ]:
if "Attribute" in deck_df and deck_df["Attribute"].dropna(how="all", axis=0).empty:
    print("No attribute data.")
else:
    attr_colors = [plot.colors_dict[attr] for attr in sorted(deck_df["Attribute"].dropna().unique())]
    _ = plot.deck_distribution(
        deck_df,
        "Attribute",
        colors=attr_colors,
        hatches=["", "XX", ".."],
    )
    plt.show()

In [ ]:
if "Attribute" in deck_df and not deck_df["Attribute"].dropna(how="all", axis=0).empty:
    display(deck_df.groupby(["Deck", "Section"])["Attribute"].value_counts().unstack(2).fillna(0))

## Secondary type

In [ ]:
if "Secondary type" in deck_df and deck_df["Secondary type"].dropna().empty:
    print("No secondary type data.")
else:
    _ = plot.deck_distribution(deck_df.explode("Secondary type"), "Secondary type")
    plt.show()

In [ ]:
if "Secondary type" in deck_df and not deck_df["Secondary type"].dropna(how="all", axis=0).empty:
    display(
        deck_df.explode("Secondary type").groupby(["Deck", "Section"])["Secondary type"].value_counts().unstack(2).fillna(0)
    )

## Monster type

In [ ]:
if "Monster type" in deck_df and deck_df["Monster type"].dropna().empty:
    print("No monster type data.")
else:
    _ = plot.deck_distribution(
        deck_df,
        "Monster type",
    )
    plt.show()

In [ ]:
if "Monster type" in deck_df and not deck_df["Monster type"].dropna(how="all", axis=0).empty:
    display(deck_df.groupby(["Deck", "Section"])["Monster type"].value_counts().unstack(2).fillna(0))

## Properties

In [ ]:
if "Property" in deck_df and deck_df["Property"].dropna().empty:
    print("No spell or trap cards.")
else:
    prop_colors = [
        plot.colors_dict["Trap Card"] if "Trap" in prop else plot.colors_dict["Spell Card"]
        for prop in sorted(deck_df["Property"].dropna().unique())
    ]
    _ = plot.deck_distribution(
        deck_df,
        "Property",
        colors=prop_colors,
        hatches=["", "XX", ".."],
    )
    plt.show()

In [ ]:
if "Property" in deck_df and not deck_df["Property"].dropna(how="all", axis=0).empty:
    display(deck_df.groupby(["Deck", "Section"])["Property"].value_counts().unstack(2).fillna(0))

## TCG & OCG Status 

In [ ]:
limit_df = check_limits(deck_df)
if not limit_df.empty:
    print("Your decks have the following forbidden, limited, and semi-limited cards:")
    display(limit_df)

In [ ]:
if "TCG status" in deck_df and deck_df["TCG status"].dropna().empty:
    print("No TCG status data.")
else:
    _ = plot.deck_distribution(deck_df, "TCG status")
    plt.show()

In [ ]:
if "TCG status" in deck_df and not deck_df["TCG status"].dropna(how="all", axis=0).empty:
    display(deck_df.groupby(["Deck", "Section"])["TCG status"].value_counts().unstack(2).fillna(0))

In [ ]:
if "OCG status" in deck_df and deck_df["OCG status"].dropna().empty:
    print("No OCG status data.")
else:
    _ = plot.deck_distribution(deck_df, "OCG status")
    plt.show()

In [ ]:
if "OCG status" in deck_df and not deck_df["OCG status"].dropna(how="all", axis=0).empty:
    display(deck_df.groupby(["Deck", "Section"])["OCG status"].value_counts().unstack(2).fillna(0))

## Archetypes & Series

In [ ]:
if "Archseries" in deck_df and deck_df["Archseries"].dropna().empty:
    print("No Archseries data.")
else:
    _ = plot.deck_distribution(deck_df.explode("Archseries"), "Archseries")
    plt.show()

In [ ]:
if "Archseries" in deck_df and not deck_df["Archseries"].dropna(how="all", axis=0).empty:
    display(deck_df.explode("Archseries").groupby(["Deck", "Section"])["Archseries"].value_counts().unstack(1).fillna(0))

## ATK & DEF

In [ ]:
if ("ATK" in deck_df or "DEF" in deck_df) and deck_df[["ATK", "DEF"]].dropna(how="all", axis=0).empty:
    print("No ATK/DEF data.")
else:
    _ = plot.deck_stem(deck_df=deck_df, y1="ATK", y2="DEF")
    plt.show()

In [ ]:
if ("ATK" in deck_df or "DEF" in deck_df) and not deck_df[["ATK", "DEF"]].dropna(how="all", axis=0).empty:
    def_df = deck_df.rename({"DEF": "Value"}, axis=1).groupby(["Deck", "Section"])["Value"].value_counts().rename("DEF")
    atk_df = deck_df.rename({"ATK": "Value"}, axis=1).groupby(["Deck", "Section"])["Value"].value_counts().rename("ATK")
    display(
        pd.merge(atk_df, def_df, how="outer", left_index=True, right_index=True)
        .unstack("Section")
        .dropna(how="all", axis=1)
        .fillna(0)
        .sort_index()
    )

## Level & Rank

In [ ]:
if "Level/Rank" in deck_df:
    level_df = (
        deck_df[["Deck", "Section", "Level/Rank", "Count"]][deck_df["Primary type"] != "Xyz Monster"]
        .dropna()
        .rename(columns={"Level/Rank": "Level"})
    )
    rank_df = (
        deck_df[["Deck", "Section", "Level/Rank", "Count"]][deck_df["Primary type"] == "Xyz Monster"]
        .dropna()
        .rename(columns={"Level/Rank": "Rank"})
    )
    if level_df.empty and rank_df.empty:
        print("No Level/Rank data.")
    else:
        _ = plot.deck_stem(deck_df=pd.concat([level_df, rank_df]), y1="Level", y2="Rank")
        plt.show()

In [ ]:
if "Level/Rank" in deck_df and not level_df.empty and not rank_df.empty:
    level_series = (
        level_df.rename(columns={"Level": "Value"}).groupby(["Deck", "Section"])["Value"].value_counts().rename("Level")
    )
    rank_series = (
        rank_df.rename(columns={"Rank": "Value"}).groupby(["Deck", "Section"])["Value"].value_counts().rename("Rank")
    )
    display(
        pd.merge(level_series, rank_series, how="outer", left_index=True, right_index=True)
        .unstack("Section")
        .dropna(how="all", axis=1)
        .fillna(0)
        .sort_index()
    )

## Link

In [ ]:
if "Link" in deck_df and deck_df["Link"].dropna().empty:
    print("No Link monsters data.")
else:
    _ = plot.deck_stem(deck_df=deck_df[deck_df["Section"] != "Main"], y1="Link")
    plt.show()

In [ ]:
if "Link" in deck_df and not deck_df["Link"].dropna().empty:
    display(deck_df.groupby(["Deck", "Section"])["Link"].value_counts().unstack(2).sort_index().fillna(0))

## Pendulum scale

In [ ]:
if "Pendulum Scale" in deck_df and deck_df["Pendulum Scale"].dropna().empty:
    print("No Pendulum monsters data.")
else:
    _ = plot.deck_stem(deck_df=deck_df, y1="Pendulum Scale")

In [ ]:
if "Pendulum Scale" in deck_df and not deck_df["Pendulum Scale"].dropna().empty:
    display(deck_df.groupby(["Deck", "Section"])["Pendulum Scale"].value_counts().unstack(2).sort_index().fillna(0))

# Check collection

In [ ]:
# Merge the collection and deck data frames
collection_df = get_collection()
if collection_df is None:
    print("No collection data to process.")
else:
    collection_df = find_cards(collection_df)
    collection_df = assign_deck(collection_df, deck_df=deck_df, return_collection=False)

Table of cards in decks missing from collection

In [ ]:
if collection_df is not None:
    display(collection_df[collection_df["missing"] > 0].dropna(how="all", axis=1))

# Epilogue

In [ ]:
benchmark(report="Deck", timestamp=timestamp)

In [ ]:
footer(timestamp)

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
export_notebook(dirs.NOTEBOOKS.user / "Deck.ipynb")

## Git

In [ ]:
git.commit("*[Dd]eck*", f"Deck update - {timestamp.isoformat()}")